# Homework 3

## FINM 37500 - 2023

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

# 1. Treasury Futures and Cheapest-to-Deliver

The file `data/fut_bond_data_FVU3_2023-04-21.xlsx` has market data on the following:
* 5-year Treasury future, expiring September 2023
* The specifications of the deliverable treasury bonds

Market quotes are provided on the futures contract and the bond prices. These will be useful for some of the analysis questions, but you do not need them for your models as you are provided a BDT tree which is fit to swaps and caps. See below for more details on this BDT model.

Suppose the present date is `2023-04-21`.

In [206]:
import pandas as pd
import numpy as np
from Binomial_Fixed import binomial, ratecurves
from Binomial_Fixed import ficcvol
from treasury_cmds import *
import scipy
from scipy.optimize import fsolve
from scipy.stats import norm
from datetime import date
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'

In [207]:
future_bonds = 'C:/Users/dcste/OneDrive/Fixed_Income_Derivatives/finm-fiderivs-2023/data/fut_bond_data_FVU3_2023-04-21.xlsx'
bdt_path = 'C:/Users/dcste/OneDrive/Fixed_Income_Derivatives/finm-fiderivs-2023/data/bdt_params_freq52_2023-04-21.xlsx'
future_description = pd.read_excel(future_bonds, sheet_name='future')
bond_info = pd.read_excel(future_bonds,sheet_name='bonds')
bdt_params = pd.read_excel(bdt_path)
Future_Price = future_description.iloc[1,1]



In [208]:
Future_Price

109.7890625

In [209]:
future_description

,field,FVU3 Comdty
0,last_update_dt,2023-04-21 00:00:00
1,px_last,109.789062
2,last_tradeable_dt,2023-09-29 00:00:00
3,fut_dlv_dt_last,2023-10-04 00:00:00
4,fut_days_expire,159
5,fut_ctd,T 3.875 11/30/27
6,fut_ctd_px,100.757812
7,fut_ctd_gross_basis,-17.074348
8,fut_ctd_net_basis,1.199828


In [210]:
bond_info['cash_price'] = bond_info['px_last'] + .5*bond_info['cpn']*(bond_info['days_acc']/(bond_info['accrued_days_between_cpn_dates']))

In [211]:
def forward_price(cash_price,cpn,repo,days_next_cpn,DAY_COUNT = 360):
    coupon = .5*(cpn)
    repo = repo*(1/100)
    inner = cash_price-coupon*np.exp(-repo*(days_next_cpn/DAY_COUNT))
    return inner*np.exp(repo*(159/360))

def carry(cpn,repo, days_acc, days_fwd,accrued_days_between_cpns,DAY_COUNT = 360):
    cpn = .5*cpn
    return (cpn*(days_fwd/accrued_days_between_cpns) - cpn*(days_acc/accrued_days_between_cpns)) - repo*(159/DAY_COUNT)



In [212]:
bond_info['forward_price'] = bond_info.apply(lambda row:forward_price(cash_price=row['cash_price'],cpn=row['cpn'],repo = row['repo_reporate'],\
    days_next_cpn = row['days_to_next_coupon']), axis = 1)
bond_info['carry'] = bond_info.apply(lambda row: carry(cpn = row['cpn'], repo=row['repo_reporate'],days_acc=row['days_acc'],days_fwd = 159,accrued_days_between_cpns=row['accrued_days_between_cpn_dates'],DAY_COUNT=360), axis=1)

In [213]:
bond_info['gross_basis'] = 32*(bond_info['px_last'] - Future_Price*bond_info['conversion'])
bond_info['net_basis'] = (bond_info['gross_basis'] - bond_info['carry'])

In [214]:
bond_info

,ticker,last_update_dt,px_last,maturity,days_to_mty,cpn,nxt_cpn_dt,days_to_next_coupon,int_acc,accrued_days_between_cpn_dates,days_acc,basis_mid,repo_implied_reporate,repo_reporate,conversion,cash_price,forward_price,carry,gross_basis,net_basis
0,91282CFZ Govt,2023-04-21,100.757812,2027-11-30,1681,3.875,2023-05-31,37,1.543613,182,145,13.138350,3.619994,4.815,0.9226,102.301425,102.530916,-1.977587,-17.074450,-15.096863
1,91282CGC Govt,2023-04-21,100.750000,2027-12-31,1712,3.875,2023-06-30,67,1.220304,181,114,17.307175,3.644369,4.815,0.9212,101.970304,102.200564,-1.644926,-12.405900,-10.760974
2,91282CGH Govt,2023-04-21,99.195312,2028-01-31,1743,3.500,2023-07-31,98,0.802486,181,83,24.180222,3.631045,4.815,0.9058,99.997799,100.382814,-1.391818,-8.051850,-6.660032
3,91282CGP Govt,2023-04-21,101.484375,2028-02-29,1772,4.000,2023-08-31,129,0.597826,184,55,32.893100,3.550495,4.815,0.9234,102.082201,102.268314,-0.996190,3.364950,4.361140
4,91282CGT Govt,2023-04-21,99.828125,2028-03-31,1803,3.625,2023-09-30,159,0.237705,183,24,38.039247,3.570506,4.815,0.9075,100.065830,100.404144,-0.789535,6.225625,7.015160


### BDT Model

In this problem you will make use of a BDT modeled binomial tree.

To save you some time, you are provided the parameters of a BDT tree fit to both swaps and caps.
* Use the file `bdt_params_freq52_2023-04-21.xlsx`
* With these $\sigma$ and $\theta$ parameters, you should be able to build a BDT tree with $T=5$ and $dt=1/52$.

#### Note
If interested in how this was done, find the data and files used to get these parameters. In particular, 
* The market quotes interpolated to weekly frequency: `cap_curves_2023-04-21_freq_52.xlsx`.
* The file to estimate the model is `Parameterize BDT.ipynb`.

## 1.1 Trading Bonds

Give brief answers to these based on the market quotes provided,
### 1.1.1
Calculate the 
* gross basis
* carry
* net basis for each bond

### 1.1.2
Which bond seems most likely to be CTD?

### 1.1.3
If you were required to put on a position today
* long one of the bonds
* short the future

which would you choose based on the data provided in the spreadsheet?

In [215]:
#1.1.1
bond_info.iloc[:,-3:]

,carry,gross_basis,net_basis
0,-1.977587,-17.074450,-15.096863
1,-1.644926,-12.405900,-10.760974
2,-1.391818,-8.051850,-6.660032
3,-0.996190,3.364950,4.361140
4,-0.789535,6.225625,7.015160


## 1.2 Conversion Factors

Calculate the conversion factor for each bond. Report it to `6` decimal places.

Do they match the conversion factor provided by Bloomberg?

In [216]:
fut_days_expire = 159
tmat = (bond_info['days_to_mty'] - 159)/365.25
conversion_factor = pd.DataFrame(ratecurves.price_bond(.06,tmat,bond_info['cpn']/(100),cpnfreq = 2,face = 100, accr_frac=0)/100, columns=['Conversion Factor'])
conversion_factor = conversion_factor.merge(bond_info['conversion'], left_index=True, right_index=True)
conversion_factor.index = bond_info['ticker']



In [217]:
conversion_factor

,Conversion Factor,conversion
ticker,,
91282CFZ Govt,0.922669,0.9226
91282CGC Govt,0.921283,0.9212
91282CGH Govt,0.905770,0.9058
91282CGP Govt,0.923408,0.9234
91282CGT Govt,0.907522,0.9075


## 1.3 BDT Tree

Report the number of steps for
* each bond's maturity
* the futures contract expiration

Build the interest-rate tree and display it.

In [218]:
FREQUENCY = 52
def number_steps(num_days, frequency):
    years = num_days/365.25
    tree_steps = round(years*frequency,6)/frequency
    return tree_steps


In [243]:
coupons = number_steps(bond_info['days_to_next_coupon'], frequency=52)
maturities = number_steps(bond_info['days_to_mty'], frequency=52)
Future_Expiry = number_steps(159,frequency=FREQUENCY)

round(maturities)

0    5.0
1    5.0
2    5.0
3    5.0
4    5.0
Name: days_to_mty, dtype: float64

- Each bond will have 5 time steps

In [244]:
Future_Expiry

0.4353182692307692

## 1.4 Bond Pricing

Use the tree to price each bond. Report
* time-0 dirty and clean price of each bond
* terminal (clean) value of each bond at futures expiration, for each state of the tree.

Thus, to report the terminal values you will need to grab the expiration column of each bond's (clean) pricing tree and adjust (inflate) it for the conversion factor.

In [221]:
bdt_params = bdt_params.set_index('maturity')

In [245]:

ratetree=  binomial.BDTtree(bdt_params['theta'],sigmas=bdt_params['fwd vol'],px_bond0=bdt_params['discount'].iloc[0], dt = 1/FREQUENCY)
ratetree.loc[:,:Future_Expiry].dropna(how = 'all')


time,0.000000,0.019231,0.038462,0.057692,0.076923,0.096154,0.115385,0.134615,0.153846,0.173077,...,0.250000,0.269231,0.288462,0.307692,0.326923,0.346154,0.365385,0.384615,0.403846,0.423077
state,,,,,,,,,,,,,,,,,,,,,
0,0.048894,0.050923,0.053074,0.055102,0.057073,0.059013,0.060938,0.062859,0.064785,0.066724,...,0.074736,0.076833,0.078980,0.081182,0.083446,0.085778,0.088183,0.090670,0.093245,0.095916
1,NaN,0.047722,0.049737,0.051638,0.053485,0.055303,0.057107,0.058908,0.060712,0.062529,...,0.070037,0.072003,0.074015,0.076079,0.078200,0.080385,0.082639,0.084970,0.087383,0.089886
2,NaN,NaN,0.046610,0.048391,0.050122,0.051827,0.053517,0.055204,0.056896,0.058598,...,0.065634,0.067476,0.069362,0.071296,0.073284,0.075331,0.077444,0.079628,0.081889,0.084235
3,NaN,NaN,NaN,0.045630,0.047262,0.048869,0.050463,0.052054,0.053649,0.055254,...,0.061889,0.063626,0.065404,0.067227,0.069102,0.071033,0.073025,0.075084,0.077217,0.079429
4,NaN,NaN,NaN,NaN,0.044666,0.046184,0.047691,0.049194,0.050701,0.052218,...,0.058488,0.060130,0.061810,0.063534,0.065305,0.067130,0.069013,0.070959,0.072974,0.075064
5,NaN,NaN,NaN,NaN,NaN,0.043673,0.045098,0.046519,0.047944,0.049379,...,0.055308,0.056860,0.058449,0.060079,0.061754,0.063480,0.065260,0.067101,0.069006,0.070983
6,NaN,NaN,NaN,NaN,NaN,NaN,0.042631,0.043975,0.045322,0.046679,...,0.052284,0.053751,0.055253,0.056794,0.058377,0.060008,0.061691,0.063431,0.065232,0.067101
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.041531,0.042803,0.044084,...,0.049378,0.050763,0.052182,0.053637,0.055133,0.056673,0.058262,0.059905,0.061607,0.063371
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.040368,0.041576,...,0.046569,0.047876,0.049213,0.050586,0.051996,0.053449,0.054948,0.056498,0.058102,0.059766


In [246]:
ratetree

time,0.000000,0.019231,0.038462,0.057692,0.076923,0.096154,0.115385,0.134615,0.153846,0.173077,...,4.826923,4.846154,4.865385,4.884615,4.903846,4.923077,4.942308,4.961538,4.980769,5.000000
state,,,,,,,,,,,,,,,,,,,,,
0,0.048894,0.050923,0.053074,0.055102,0.057073,0.059013,0.060938,0.062859,0.064785,0.066724,...,3.307579e+08,3.704113e+08,4.144761e+08,4.633889e+08,5.176224e+08,5.776875e+08,6.441355e+08,7.175603e+08,7.986006e+08,8.880333e+08
1,NaN,0.047722,0.049737,0.051638,0.053485,0.055303,0.057107,0.058908,0.060712,0.062529,...,3.099639e+08,3.471245e+08,3.884190e+08,4.342567e+08,4.850807e+08,5.413697e+08,6.036403e+08,6.724490e+08,7.483946e+08,8.322048e+08
2,NaN,NaN,0.046610,0.048391,0.050122,0.051827,0.053517,0.055204,0.056896,0.058598,...,2.904773e+08,3.253016e+08,3.640000e+08,4.069561e+08,4.545849e+08,5.073351e+08,5.656909e+08,6.301738e+08,7.013448e+08,7.798862e+08
3,NaN,NaN,NaN,0.045630,0.047262,0.048869,0.050463,0.052054,0.053649,0.055254,...,2.739020e+08,3.067392e+08,3.432294e+08,3.837343e+08,4.286453e+08,4.783855e+08,5.334114e+08,5.942147e+08,6.613246e+08,7.353842e+08
4,NaN,NaN,NaN,NaN,0.044666,0.046184,0.047691,0.049194,0.050701,0.052218,...,2.588525e+08,2.898854e+08,3.243707e+08,3.626500e+08,4.050934e+08,4.521006e+08,5.041031e+08,5.615656e+08,6.249881e+08,6.949785e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.123213e-11,3.482458e-11,3.879422e-11,4.317559e-11,4.801068e-11
257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.184785e-11,3.547818e-11,3.948504e-11,4.390684e-11
258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.236580e-11,3.602115e-11,4.005504e-11


In [223]:
ratetree.head()

time,0.000000,0.019231,0.038462,0.057692,0.076923,0.096154,0.115385,0.134615,0.153846,0.173077,...,4.826923,4.846154,4.865385,4.884615,4.903846,4.923077,4.942308,4.961538,4.980769,5.000000
state,,,,,,,,,,,,,,,,,,,,,
0,0.048894,0.050923,0.053074,0.055102,0.057073,0.059013,0.060938,0.062859,0.064785,0.066724,...,3.307579e+08,3.704113e+08,4.144761e+08,4.633889e+08,5.176224e+08,5.776875e+08,6.441355e+08,7.175603e+08,7.986006e+08,8.880333e+08
1,NaN,0.047722,0.049737,0.051638,0.053485,0.055303,0.057107,0.058908,0.060712,0.062529,...,3.099639e+08,3.471245e+08,3.884190e+08,4.342567e+08,4.850807e+08,5.413697e+08,6.036403e+08,6.724490e+08,7.483946e+08,8.322048e+08
2,NaN,NaN,0.046610,0.048391,0.050122,0.051827,0.053517,0.055204,0.056896,0.058598,...,2.904773e+08,3.253016e+08,3.640000e+08,4.069561e+08,4.545849e+08,5.073351e+08,5.656909e+08,6.301738e+08,7.013448e+08,7.798862e+08
3,NaN,NaN,NaN,0.045630,0.047262,0.048869,0.050463,0.052054,0.053649,0.055254,...,2.739020e+08,3.067392e+08,3.432294e+08,3.837343e+08,4.286453e+08,4.783855e+08,5.334114e+08,5.942147e+08,6.613246e+08,7.353842e+08
4,NaN,NaN,NaN,NaN,0.044666,0.046184,0.047691,0.049194,0.050701,0.052218,...,2.588525e+08,2.898854e+08,3.243707e+08,3.626500e+08,4.050934e+08,4.521006e+08,5.041031e+08,5.615656e+08,6.249881e+08,6.949785e+08


In [247]:
FV = 100
compound = FREQUENCY
dt = 1/compound
cpn_freq = 2

In [225]:
terminal_values = pd.DataFrame(index =ratetree.index, columns=bond_info.index, dtype=float)
px_bonds = pd.DataFrame(dtype=float, index =bond_info.index , columns=['clean_price'])

In [249]:
for idx, bond in enumerate(bond_info.index):
    time_steps = round(maturities[idx]/dt)
    cpns = bond_info.loc[bond,'cpn']*(1/100)

    wrapper_function = lambda rate : binomial.payoff_bond(rate, dt, facevalue=FV*(1+cpns/cpn_freq))

    cftree = binomial.construct_bond_cftree(maturities[idx], compound = compound, cpn = cpns)

    bondtree = binomial.bintree_pricing(payoff=wrapper_function, ratetree=ratetree.iloc[:time_steps,:time_steps],cftree=cftree)
    accrued_int_tree = binomial.construct_accinttree(cftree=cftree,compound=compound,cpn=cpns)
    tree = np.maximum(bondtree - accrued_int_tree,0)
    px_bonds.loc[bond] = tree.iloc[0,0]
    terminal_values[bond] = tree[round(Future_Expiry)]


In [250]:
terminal_values.dropna(inplace = True)
terminal_values

,0,1,2,3,4
state,,,,,
0,102.280239,101.932095,99.949739,101.936259,99.895311


In [240]:
ts = bondtree.columns

In [241]:
ts

Float64Index([     0.0, 0.019231, 0.038462, 0.057692, 0.076923, 0.096154,
              0.115385, 0.134615, 0.153846, 0.173077,
              ...
                  4.75, 4.769231, 4.788462, 4.807692, 4.826923, 4.846154,
              4.865385, 4.884615, 4.903846, 4.923077],
             dtype='float64', name='time', length=257)

## 1.5 CTD

Use your terminal values calculated above to state which bond is CTD in each interest-rate state (at this expiration node.) 

Report the duration of each bond (as of today's price, not recomputed for the interest-rate nodes.) Do you see a relationship between the time-0 duration and the at-expiration CTD?

## 1.6 Futures Price

Model the futures price with the tree approach.
* Use the CTD terminal value for each rate.
* Step backward through the tree.

As you step backward remember that for a futures contract
* no discounting by the riskfree rate
* the futures contract has no capital requirement and thus an expected P&L of zero under this measure.

Thus, each node is the simple average of the two nodes at the following step.

**Report the futures price.**

### Compare
How does it compare to 
* the quoted futures price
* the modeled bond prices

## 1.7 Early Delivery
**Optional**
Above we modeled the terminal value at the futures expiration. Now consider if early delivery would be better.

Which periods in the tree are eligible to deliver based on the parameters of the 5-year futures contract? 

Based on your model, does it make sense to deliver early in any of the nodes of the tree?

## 1.8 Option-Adjusted Spread
**Optional**

Calculate and report the option-adjusted spread (OAS) for the future.

Note that you
* do NOT need to recalculate the bond prices
* will simply add a constant rate (at every node) for discounting the futures price in the previous problem.

What does the OAS indicate?

***

# 2. Fed Funds Futures

The file `data/fedfutures_2023-04-21.xlsx`jj has market data on the following:
* Fed Fund Futures Chain out 18 months.
* Dates of upcoming Fed meetings (approximated in 2024.)
* Spot Fed Funds data
* Prices of the futures chain on a historic date.

Suppose the present date is `2023-04-21`.

## 2.1 Chart the Fed Futures Rates

Chart the Fed Funds curve at
* the present date
* the historic date

Note that you are charting the implied Fed Funds Futures *rate*, not price.

Comment on how today's **open interest** varies across the chain.

## 2.2 Extracting the Expected Path of Fed Funds Rates

The Fed has a great deal of control over the Fed Funds Rate. We simplify by assuming the Fed 
* sets the rate exactly at its list of meeting dates.
* does not change the rate betweeen meeting dates.

Use the present data to calculate--and plot--the implied set of expected Fed Funds rates as of each meeting date.

#### Note
One (minor) assumption:
* Consider months, $t$, where there is a meeting, but such that in month $t+1$ there is no meeting.
* There will be two reasonable ways to extract the expected fed funds rate:
    1. Use the futures rate from the $t+1$-contract
    2. Calculate the implied rate for the remainder of month $t$, knowing the expected rate at the end of month $t+1$.
* These are both reasonable and will likely not differ much.
* Here, use the simpler method #1--that is, for months with no meeting in the following month, the calculation is very simple.

## 2.3 Compare to the Historic Curve

Use the price data in the historic tab to extract the expectations at the previous date.
* Note that you do not need to "bootstrap" up from the historic date to the current date. 
* There was no meeting in the current month, so its futures price is enough to get started.

Compare this to the answer in the previous problem, for the current data.

## 2.4 Analyzing the Expected Path

These questions are both conceptual--no calculation required.

### 2.4.1
Conceptually, is the path extracted above the **expected path**? In what sense is it or is it not?

### 2.4.2

Probability Distributions

The implied path above is not representative of any single actual path of Fed rates, which are typically changed by 25bps at a time.

Conceptually, what would you need to make probability statements about the Fed moving rates up/down by 25bps on any given meeting date? For instance, as seen in the `probabilities` tab of the [CME FedWatch Tool](https://www.cmegroup.com/markets/interest-rates/cme-fedwatch-tool.html)?

Do not quantitatively solve this--just a conceptual answer is fine.

***